In [ ]:
#First, let's set up the environment by importing the necessary libraries and defining our workspace, to avoid adding whole file paths every time we reference a feature. 
#We'll also define the output geodatabase and the aprx
import arcpy
import os
import geopandas
arcpy.env.workspace = r"G:\My Drive\INFO612_Advanced_GIS\Lab 1 (1)\Lab1_NYTeens"
output_gdb = r"G:\My Drive\INFO612_Advanced_GIS\Lab 1 (1)\Lab1_NYTeens\lab1_nyteens.gdb" 
aprx = arcpy.mp.ArcGISProject('CURRENT')

In [ ]:
#Now we define the map and map object, so we can add features to the map.
maps = aprx.listMaps("NYTeensMap")
if maps:
    map_obj = maps[0]
else:
    print("Map 'NYTeensMap' not found. Using first available map instead.")
    map_obj = aprx.listMaps()[0]

In [ ]:
#We're going to add the features that are in JSON (libraries and parks) format to the map
arcpy.conversion.JSONToFeatures(
    in_json_file=r"G:\My Drive\INFO612_Advanced_GIS\Lab 1 (1)\Lab1_NYTeens\Data\LIBRARY_20250926.geojson",
    out_features=r"G:\My Drive\INFO612_Advanced_GIS\Lab 1 (1)\Lab1_NYTeens\Default.gdb\Library_Locations",
    geometry_type="POINT"
)
arcpy.conversion.JSONToFeatures(
    in_json_file=r"G:\My Drive\INFO612_Advanced_GIS\Lab 1 (1)\Lab1_NYTeens\Data\Parks Properties_20250926.geojson",
    out_features=r"G:\My Drive\INFO612_Advanced_GIS\Lab 1 (1)\Lab1_NYTeens\Default.gdb\Parks_Locations",
    geometry_type="POLYGON",
)

In [ ]:
#convert the data that's in table format to points to add to the map -- middle and high school locations and 'youth services' designated facilities run by the city and its partners
arcpy.management.XYTableToPoint(
    in_table="Schools_FacDB_20251001.csv",
    out_feature_class=r"G:\My Drive\INFO612_Advanced_GIS\Lab 1 (1)\Lab1_NYTeens\Default.gdb\Schools_Locations",
    x_field="longitude",
    y_field="latitude",
    z_field=None,
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')
arcpy.management.XYTableToPoint(
    in_table="YouthServices_FacDB_20251001.csv",
    out_feature_class=r"G:\My Drive\INFO612_Advanced_GIS\Lab 1 (1)\Lab1_NYTeens\Default.gdb\YouthServices_Locations",
    x_field="longitude",
    y_field="latitude",
    z_field=None,
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision'
)
#add privately owned public spaces points (POPS) using the same tool
arcpy.management.XYTableToPoint(
    in_table="Privately_Owned_Public_Spaces__POPS__20250926.csv",
    out_feature_class=r"G:\My Drive\INFO612_Advanced_GIS\Lab 1 (1)\Lab1_NYTeens\Default.gdb\POPS_Locations",
    x_field="Longitude",
    y_field="Latitude",
    z_field=None,
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision'
)

In [ ]:
#there is an inordinate amount of 'youth services' locations, and looking at the attribute table, many of them are not actually serving the demographic we're interested in, so we'll select by attribute to only include the points that concern 6-12th graders
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="YouthServices_Locations",
    selection_type="NEW_SELECTION",
    where_clause="factype = 'AFTERSCHOOL PROGRAMS; EDUCATIONAL SERVICES' Or factype = 'BEACON' Or factype = 'COMPASS EXPLORE' Or factype = 'COMPASS HIGH' Or factype = 'CORNERSTONE' Or factype = 'SCHOOL''S OUT NEW YORK CITY (SONYC)' Or factype = 'YOUTH RECREATIONAL SERVICES/YOUTH ATHLETIC LEAGUES' Or factype = 'EDUCATIONAL SUPPORT: HIGH SCHOOL YOUTH'",
    invert_where_clause=None
)
#then create a layer from the selected features. This will be our new Youth Services Locations layer.